In [1]:
import csv 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SpotifySongs").getOrCreate()

df = spark.read.csv("../assets/data/spotify-2023.csv", header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 14:49:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/usr/local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [41]:
df.show()

+--------------------+--------------------+------------+-------------+--------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+----------------+---+----+-----+--------------+---------+--------+--------------+------------------+----------+-------------+
|          track_name|      artist(s)_name|artist_count|released_year|released_month|released_day|in_spotify_playlists|in_spotify_charts|   streams|in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|bpm| key| mode|danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|
+--------------------+--------------------+------------+-------------+--------------+------------+--------------------+-----------------+----------+------------------+---------------+-------------------+----------------+----------------+---+----+-----+--------------+---------+--------+--------------+-------

1. Most Streamed & Highest Ranking Tracks:

- Identify the top 5 tracks with the highest number of streams in 2023.
- Identify the top 5 ranking tracks across all platforms.
- Is there any crossover between these two findings?

In [39]:
top_5_streams = df.select("track_name", "artist(s)_name", "streams").orderBy(df.streams.cast('int').desc()).limit(5)

top_5_streams.show()


+-----------------+--------------+----------+
|       track_name|artist(s)_name|   streams|
+-----------------+--------------+----------+
|Take Me To Church|        Hozier|2135158446|
|          Circles|   Post Malone|2132335812|
|    Love Yourself| Justin Bieber|2123309722|
|        All of Me|   John Legend|2086124197|
|   Counting Stars|   OneRepublic|2011464183|
+-----------------+--------------+----------+

